<a href="https://colab.research.google.com/github/leejunho12316/HonGongMachine/blob/main/%ED%98%BC%EA%B3%B5%EB%A8%B8%EC%8B%A0_%EB%B3%B5%EC%8A%B5/%ED%98%BC%EA%B3%B5%EB%A8%B8%EC%8B%A0_5_2_%EB%B3%B5%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
wine = pd.read_csv("https://bit.ly/wine_csv_data")
wine_input = wine[['alcohol','sugar','pH']].to_numpy()
wine_target = wine['class'].to_numpy()

In [16]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(wine_input, wine_target, test_size = 0.2, random_state = 42)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.25, random_state = 42)

In [17]:
print(sub_input.shape, test_input.shape, val_input.shape)

(3897, 3) (1300, 3) (1300, 3)


In [18]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(test_input, test_target))
print(dt.score(val_input, val_target))

0.9969207082371054
0.8423076923076923
0.86


교차 검증

In [38]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
import numpy as np

def print_scores(scores):
  print("fit_time: ", scores['fit_time'])
  print("score_time: ", scores['score_time'])
  print("test_score: ", scores['test_score'])
  print("평균 점수 : ", np.mean(scores['test_score']))
  print()

splitter = StratifiedKFold(n_splits = 30, shuffle=True, random_state = 42)

print("원본 데이터로 교차검증")
scores = cross_validate(dt, wine_input, wine_target, cv = splitter)
print_scores(scores)

print("훈련 데이터로 교차검증")
scores = cross_validate(dt, sub_input, sub_target, cv = splitter)
print_scores(scores)

print("테스트 데이터로 교차검증")
scores = cross_validate(dt, test_input, test_target, cv = splitter)
print_scores(scores)

print("검증 데이터로 교차검증")
scores = cross_validate(dt, val_input, val_target, cv = splitter)
print_scores(scores)

원본 데이터로 교차검증
fit_time:  [0.0183835  0.01951265 0.01886559 0.01800656 0.01846576 0.0179224
 0.01794362 0.01804018 0.01789188 0.01771665 0.02099156 0.01811147
 0.01816821 0.01731873 0.02057934 0.01765013 0.0176127  0.01745415
 0.01784205 0.01780224 0.01976919 0.02316642 0.01884127 0.02109218
 0.01773596 0.01781249 0.01813364 0.01774359 0.01776266 0.01827502]
score_time:  [0.00161815 0.00183344 0.00170708 0.00180817 0.00174665 0.00168872
 0.00169182 0.00174785 0.00151587 0.00171494 0.00159669 0.00166106
 0.00166202 0.00161648 0.00161886 0.00166631 0.00172687 0.0017705
 0.00162339 0.00162196 0.00182462 0.00162601 0.00164127 0.00158548
 0.00169063 0.00168562 0.00178099 0.00163484 0.0028646  0.00160122]
test_score:  [0.87557604 0.88479263 0.86635945 0.88479263 0.90322581 0.85253456
 0.9124424  0.86175115 0.87557604 0.87557604 0.86175115 0.88940092
 0.86175115 0.86635945 0.84331797 0.88940092 0.87096774 0.84722222
 0.93055556 0.87962963 0.85648148 0.87037037 0.88888889 0.88888889
 0.89351852 

하이퍼파라미터 튜닝

In [55]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

#그리드 서치 파라미터 한 개
dt = DecisionTreeClassifier(random_state = 42)
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)

params = {'min_impurity_decrease' : [0.001, 0.002, 0.003, 0.004, 0.005]}
gs = GridSearchCV(dt, params, n_jobs = -1, cv = splitter)
gs.fit(train_input, train_target)


print("all_scores : ", gs.cv_results_['mean_test_score'])
print("best_parameter ; ", gs.best_params_)
dt = gs.best_estimator_
print("score : ", dt.score(train_input, train_target))

all_scores :  [0.86088076 0.85375908 0.85240996 0.85029235 0.84875389]
best_parameter ;  {'min_impurity_decrease': 0.001}
score :  0.8724263998460651


In [56]:
#그리드 서치 파라미터 여러 개
dt = DecisionTreeClassifier(random_state = 42)
spltiter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)

params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0011),
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)}
gs = GridSearchCV(dt, params, n_jobs = -1, cv = splitter)
gs.fit(train_input, train_target)

print("all_scores : ", gs.cv_results_['mean_test_score'])
print("best_parameter : ", gs.best_params_)
dt = gs.best_estimator_
print("score : ", dt.score(train_input, train_target))

all_scores :  [0.85395139 0.85395139 0.85395139 0.85414406 0.85433674 0.85452942
 0.85452942 0.85472173 0.85472173 0.85510708 0.85741292 0.85702794
 0.85702868 0.85702868 0.85799096 0.85837706 0.85895472 0.85857122
 0.85857085 0.8589562  0.85895472 0.85895472 0.85876278 0.86030273
 0.86184193 0.86242033 0.86280569 0.86242219 0.86261375 0.86299911
 0.85895991 0.85915333 0.85799837 0.8591511  0.86184452 0.86242256
 0.86261561 0.86184749 0.86261598 0.86300133 0.85895731 0.85684156
 0.85530199 0.85838039 0.86049652 0.86088224 0.8612676  0.86088484
 0.86165333 0.86184638 0.86280643 0.8560727  0.85549578 0.85934526
 0.86165296 0.8622321  0.86126908 0.86069364 0.86146213 0.86107789
 0.86049948 0.85645509 0.85587891 0.85953609 0.86107455 0.86107677
 0.86107603 0.86011598 0.86126871 0.86088447 0.86684638 0.85895768
 0.85664888 0.8603068  0.86280754 0.8630017  0.86126908 0.85992367
 0.8608841  0.86069216 0.86434489 0.85761153 0.85703498 0.85953831
 0.86223099 0.86242515 0.86049985 0.85934675 0.8

In [74]:
from scipy.stats import uniform, randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold

params = {
    'min_impurity_decrease' : uniform(0.0001, 0.001),
    'max_depth' : randint(20,50),
    'min_samples_split' : randint(2,25),
    'min_samples_leaf' : randint(1,25)
}

splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)

dt = DecisionTreeClassifier(random_state = 42)
rs = RandomizedSearchCV(dt, params, n_iter = 100, n_jobs = -1, random_state = 42, cv=splitter)
rs.fit(train_input, train_target)

print("all_scores : ", rs.cv_results_['mean_test_score'])
print("best_parameter : ", rs.best_params_)
dt = rs.best_estimator_
print("score : ", dt.score(train_input, train_target))

all_scores :  [0.86107344 0.86165185 0.86472914 0.86183971 0.8608815  0.86434378
 0.8616511  0.86145917 0.86318994 0.86396139 0.86165147 0.86107344
 0.86280495 0.86260857 0.8610727  0.86125982 0.86376723 0.86492293
 0.86010968 0.86241737 0.86164555 0.86165036 0.86241922 0.86222729
 0.86203461 0.86549726 0.86318957 0.86010857 0.86222988 0.86338224
 0.86357603 0.86318882 0.86261338 0.86491663 0.86453424 0.86280569
 0.8622284  0.86318549 0.86068845 0.86145954 0.86145768 0.85972729
 0.86088224 0.86088113 0.86068771 0.86299615 0.86261227 0.86126538
 0.86088039 0.86280532 0.86222914 0.86165036 0.86242145 0.86145991
 0.86184267 0.86184415 0.86280606 0.86010931 0.86088076 0.86261338
 0.86646139 0.86107307 0.86184267 0.86184378 0.86049689 0.86030458
 0.86280569 0.86376575 0.86280569 0.86107307 0.86126575 0.85837743
 0.86376797 0.8601119  0.8631892  0.86010857 0.86107529 0.86357566
 0.86164888 0.86145768 0.86338261 0.86626464 0.8651119  0.86107344
 0.86030162 0.86184045 0.86049689 0.85914629 0.8

In [ ]:
all_scores :  [0.86511513 0.86261235 0.86838528 0.86588547 0.86376731 0.86434497
 0.86280503 0.86280484 0.86357592 0.86357555 0.86280503 0.8626142
 0.86472977 0.86954283 0.86203543 0.86761827 0.86222884 0.86473033
 0.86877082 0.86184423 0.86126657 0.86511494 0.8626142  0.86203543
 0.86511476 0.86607722 0.86222773 0.86684682 0.86261309 0.86338436
 0.8629977  0.86242171 0.86184478 0.86165211 0.86049808 0.86530706
 0.86280521 0.86684775 0.86203524 0.86318983 0.86780947 0.86761624
 0.86126694 0.86934867 0.86857889 0.86530743 0.86434497 0.86415303
 0.86838602 0.86530688 0.86145813 0.86684626 0.8618446  0.86145961
 0.86338454 0.86569131 0.86242152 0.86376805 0.86203543 0.86376916
 0.86511457 0.86184275 0.86338454 0.86242004 0.86107481 0.86203654
 0.86184478 0.86434552 0.86184478 0.86338473 0.86299993 0.8641534
 0.86338269 0.85972662 0.86415303 0.86665433 0.86261253 0.86222884
 0.86858111 0.86472903 0.86242097 0.86261457 0.86742448 0.86434497
 0.86684682 0.86184423 0.86107481 0.86877193 0.86338362 0.86242171
 0.8674243  0.86222773 0.86242004 0.86415414 0.86376805 0.86261272
 0.86184478 0.86357518 0.86203635 0.86203691]
best_parameter :  {'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
score :  0.8928227823744468